# Préparation

In [ ]:
# Clonage du repo GitHub et installation des prérequis
!git clone https://github.com/ultralytics/yolov5
!pip install -U -r yolov5/requirements.txt

In [ ]:
# Installation de certaines versions de PyTorch et Torchvision pour utiliser le GPU (optionnel)
# !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
%cd /content/yolov5

In [ ]:
import torch
print('On utilise PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'Pas de GPU disponible'))

# Téléchargement Drive (inutile)

In [ ]:
import gdown

url = 'https://drive.google.com/uc?id=1qfaf-ihX_al_QnLaGrXCTl7MMg97wr5K'
output = 'Archive.zip'
gdown.download(url, output, quiet=False)

In [ ]:
from zipfile import ZipFile

with ZipFile('Archive.zip', 'r') as zip:
    zip.extractall()

In [ ]:
rm Archive.zip

# Importation Drive (inutile)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# Téléchargement de la BDD sur Roboflow

In [ ]:
!curl -L "https://app.roboflow.com/ds/cRe8I079jh?key=JUjvkJjR9h" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

# Train

In [ ]:
# Modification de data.yaml pour fonctionner sur colab
import os
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/data.yaml

train: ./train/images
val: ./valid/images

nc: 1
names: ['person']

In [ ]:
# Installation de WandB pour le suivi de l'entrainement
!pip install wandb

In [ ]:
!gdown https://drive.google.com/uc?id=1KI87zpm44JmIwoPShwrpSln5z-E2OJlz

In [ ]:
# Entrainement du modèle
# On rentre des images en 640x640
# Batch : 64 pour S ; 32 pour M ; 16 pour L ; 8 pour X
# Epochs : 100 - 300
# On choisit des poids déjà entrainés sur coco
# Les nouveaux poids seront stockés ici /content/yolov5/runs/exp/weights/best.pt
%cd /content/yolov5/
!python train.py --img 640 --batch 64 --epochs 50 --data data.yaml --weights yolov5s.pt --cfg yolov4-tiny.yaml

# Test

In [ ]:
# On essaye de détecter sur les images pour tester les poids
!python detect.py --source /content/yolov5/valid/images --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 

In [ ]:
# On teste la précision des poids
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data data.yaml --img 640